In [5]:
#import basic tools

import pandas as pd
import numpy as np
import biom
from biom.util import biom_open 

import matplotlib.pyplot as plt
import seaborn as sns
import pathlib

import qiime2 as q2
from skbio import TreeNode

import os
import shutil
from shutil import copyfile, rmtree
import tempfile
from zipfile import ZipFile

In [6]:
#extra set of function sfor the emperor hack

def zip_dir(qzv_name,directory):
    zf = ZipFile(qzv_name, "w")
    for dirname, subdirs, files in os.walk(directory):
        zf.write(dirname)
        for filename in files:
            zf.write(os.path.join(dirname, filename))
    zf.close()
    
def assign_emperor_settings(qzv_name):
    emp=q2.Visualization.load(qzv_name)
    uuid =str(emp.uuid)
    qzv= ZipFile(qzv_name,'r')
    qzv.extractall()
    qzv.close()
    controller_path =uuid+'/data/js'
    
    #assumes controller.js with injected code is in emperor_hack directory
    copyfile('./emperor_hack/controller.js',controller_path+'/controller.js')
    zip_dir(qzv_name,uuid)
    rmtree(uuid)

In [7]:
#shared functions

#function to denote the individual in the the column that will later be renamed 'your_sample'
def note_self(sample,col):
    if sample == col:
        return 'You'
    else:
        return 'Other AG Participants'

#fucntion to change format from Qiime-style #sampleid to Qiita-style sample_name
def resolve_sample_name(qiime_df):
    try:
        qiime_df.rename(index=str, columns={"#sampleid": "sample_name"}, inplace=True)
    except:
        print('#sampleid not found')
    return qiime_df

#function to deal with the missing zeroes and qiita study ID
#assume that if the id starts with 10317 it is correctly formatted, otherwise add the zeros and QiitaID(10317)
def append_qiita_zeroes(agp_df):
    agp_df['sample_name'] = agp_df['sample_name'].apply(lambda x: '10317.'+(len(str(x))-5)*'0'+str(x)[-5:len(str(x))] if not str(x).startswith('10317.') else str(x))
    return agp_df

#shorthand function to always clean up the metadata to fix the #sampleid, zeroes addition, and null value remapping
def clean_panda(file):
    cp_df = pd.read_csv(file,header=0, sep ='\t', decimal = ',',dtype={'sample_name': str,'#SampleID':str},
            true_values =['true','yes','y','Yes','Y','YES'],
            false_values=['false','no','n','No','N','NO'],
            na_values=['Unknown','Unspecified','no_data','not applicable','Missing: not collected', 'Missing: not provided'],
            low_memory= False
            )
    #cast all metadata categories to lower for easier reference
    cp_df.columns = cp_df.columns.str.lower()
    
    #align dataframes on 'sample_name'
    cp_df = resolve_sample_name(cp_df)
    cp_df = append_qiita_zeroes(cp_df)
    cp_df.set_index('sample_name', inplace=True)
    return cp_df

#simple function to avoid copy-pasting save_biom command
def save_biom(biom_filename, biom_table):
    with biom_open(biom_filename, 'w') as f:  
        biom_table.to_hdf5(f, "Q2-2018.2")
    print("Saved to " + biom_filename)

#function to clean up biom tables to ensure that no features without samples or samples without features are included
#option to save cleaned biom tables as well as accept either biom-format objects or filenames
def clean_biom(biom_to_clean,save=True):
    if not isinstance(biom_to_clean,biom.Table):
        clean_biom = biom.load_table(biom_to_clean)
    else:
        clean_biom = biom_to_clean
        
    #filter to remove any features that are not found in any sample or samples with no features
    clean_biom.remove_empty(inplace=True)
    
    if save:
        #save cleaned biom
        save_biom(biom_to_clean.replace('.biom','')+'_positive_only.biom',clean_biom)
    return clean_biom

#function to write out the metadata both for the single individuals as well as the custom merged metadata with their
#sample highlighted
def save_individual_metadata(md_with_ids,ids):
    for sample in ids:
        #create path if it doesn't exist
        md_folder = './individual_reports/'+sample+'/raw/metadata/'
        
        pathlib.Path(md_folder).mkdir(parents=True, exist_ok=True)        

        #highlight the individual 
        temp_series = pd.Series(ids,index=ids).apply(lambda x:note_self(x,sample))
        temp_series.fillna('Other AG Participants',inplace=True)
        #could abstract this if desired
        md_with_ids['your_sample'] = temp_series        
        
        #save metadata of individual highlighted on others
        md_with_ids.to_csv(md_folder +sample + '_metadata_merged.tsv',index=True, index_label='sample_name',sep='\t')
        
        #save metadata of individual
        #individual_metadata = individual_merged_metadata.filter(like=sample, axis=0)
        md_with_ids.to_csv(md_folder +sample +  '_metadata_only.tsv',index=True, index_label='sample_name',sep='\t')
        

def save_individual_bioms(ids,biom_to_save,suffix=''):
    #this will save one biom file with the individual in it and one biom with only the idividual
    for sample in ids:
        #create path if it doesn't exist
        biom_folder = './individual_reports/'+sample+'/raw/biom/'+suffix
        tsv_folder = './individual_reports/'+sample+'/raw/tsv/'
        
        pathlib.Path(biom_folder).mkdir(parents=True, exist_ok=True)
        pathlib.Path(tsv_folder).mkdir(parents=True, exist_ok=True)
            
        #save biom with individual noted
        save_biom(biom_folder +'/'+sample + '_' + suffix +'_merged.biom',biom_to_save)
        
        #filter biom to leave only individual
        ind_biom = biom_to_save.filter([sample],inplace=False).remove_empty()
        biom_name = biom_folder +'/'+sample + '_' + suffix + '_individual.biom'
        save_biom(biom_name,ind_biom)
        
        #want to save as flat file for users to download
        #Daniel suggested this but it does not directly save to file so leave for troubleshooting later #biom_to_save.to_tsv(tsv_folder + '/'+sample + '_' + suffix + '_individual.tsv')
        #cli call out for now
        tsv_name = tsv_folder + '/'+sample + '_' + suffix + '_individual.tsv'
        !biom convert \
            -i $biom_name \
            -o $tsv_name \
            --to-tsv

def import_bioms(ids,suffix):
    for sample in ids:
        #create path if it doesn't exist
        biom_folder = './individual_reports/'+sample+'/raw/biom/'+suffix
        qza_folder = './individual_reports/'+sample+'/raw/qza/'+suffix
               
        pathlib.Path(qza_folder).mkdir(parents=True, exist_ok=True)        
                
        input_biom = biom_folder+'/' +sample + '_' + suffix + '_individual.biom'
        output_qza = qza_folder +'/'+sample + '_' + suffix + '_individual.qza'
        
        
        !qiime tools import \
            --input-path $input_biom \
            --type 'FeatureTable[Frequency]' \
            --source-format BIOMV210Format \
            --output-path $output_qza
    
        print("Imported " + sample + " to " + output_qza)
        
def summarize_qza(ids,suffix):
     for sample in ids:
         #create path if it doesn't exist
        qzv_folder = './individual_reports/'+sample+'/qzv/'+suffix
        pathlib.Path(qzv_folder).mkdir(parents=True, exist_ok=True)
        
        qza_folder = './individual_reports/'+sample+'/raw/qza/'+suffix
        input_qza = qza_folder + '/'+sample + '_' + suffix + '_individual.qza'
        output_qzv = qzv_folder + '/'+sample + '_' + suffix + '_individual.qzv'
        ind_metadata = './individual_reports/'+sample+'/raw/metadata/' + sample + '_metadata_only.tsv'
        
        print("Summarizing " + sample + " to " + output_qzv) 
               
        !qiime feature-table summarize \
            --i-table $input_qza \
            --m-sample-metadata-file $ind_metadata \
            --o-visualization $output_qzv
        
        print('finished ' + sample)

In [8]:
#create the taxonomy and tree
def create_tree(biom_to_tree,tree_name,create_qza=True):

    taxonomy = []

    for feature in biom_to_tree.ids(axis='observation'):
        taxonomy.append((str(feature),biom_to_tree.metadata(feature,axis='observation')['taxonomy']))
    
    print("taxonomy made")
    #now create the tree
    tree = TreeNode.from_taxonomy(taxonomy)
    
    for n in tree.traverse():
        n.length = 1

    #print("tree made, rooting")
    # look for tree.root in TreeNode
    #root it; this appeared to be causing issues so disabling for use of beta-phylogenetic-alt
    #stack = tree.children
    #while len(stack) > 2:
    #    ind = stack.pop()
    #    intermediate = TreeNode()
    #    intermediate.length = 0.0
    #    intermediate.extend(stack)
    #    tree.append(intermediate)
    #    for k in stack:
    #        tree.remove(k)
    #    tree.extend([ind, intermediate])
    
    print("tree complete, starting checks")
    
    #check that the biom is compatible
    try:
        assert set(biom_to_tree.ids(axis='observation')).issubset({n.name for n in tree.tips()})
    except:
        print("failed to find ids")
        pass
    
    try:
        assert (biom_to_tree.sum('observation') > 0).all()
    except:
        print("some observations not positive")
        create_qza = False
        pass
    try:
        assert (biom_to_tree.sum('sample') > 0).all()
    except:
        print("some samples have no features")
        create_qza = False
        pass
        
    try:
        #write it out
        tree_nwk = tree_name +'.nwk'
        tree.write(tree_nwk)

        if create_qza:
            qza_rooted = tree_name + '.qza'
            
            !qiime tools import \
                --input-path $tree_nwk \
                --type 'Phylogeny[Rooted]' \
                --output-path $qza_rooted
    except:
        print('Error in tree creation')

In [9]:
def add_taxonomy(biom_to_add_tax):
    full_features = biom_to_add_tax.ids(axis='observation')
    ordered_obs_dict = {}
    for row in full_features:
        feature_list = row.split('|',7)
        for item in feature_list:
            if 'd__' in item:
                ordered_obs_dict.update({row : {'Kingdom':item.replace('d__','')}})
            elif 'k__' in item:
                ordered_obs_dict.update({row : {'Kingdom':item.replace('k__','')}})
            elif 'p__' in item:
                ordered_obs_dict[row].update({'Phylum':item.replace('p__','')})
            elif 'c__' in item:
                ordered_obs_dict[row].update({'Class':item.replace('c__','')})
            elif 'o__' in item:
                ordered_obs_dict[row].update({'Order':item.replace('o__','')})
            elif 'f__' in item:
                ordered_obs_dict[row].update({'Family':item.replace('f__','')})
            elif 'g__' in item:
                ordered_obs_dict[row].update({'Genus':item.replace('g__','')})
            elif 's__' in item:
                ordered_obs_dict[row].update({'Species':item.replace('s__','')})
            else:
                print(feature_list)
                print(item)   

    counter_dict = {'Kingdom':1,'Phylum':1,'Class':1,'Order':1,'Family':1,'Genus':1,'Species':1}

    for feature in ordered_obs_dict:
        for key in counter_dict:
            if key in ordered_obs_dict[feature]:
                temp =ordered_obs_dict[feature][key]
            else:
                ordered_obs_dict[feature].update({key: key+'_unreported_'+str(counter_dict[key])})
                counter_dict[key] = counter_dict[key] +1

    obs_dict = {}
    for key in ordered_obs_dict.keys():
        temp_list = []
        #temp_list.append(aligned_dict[key]['Domain']) #dropping domain to prefer kingdom
        temp_list.append(ordered_obs_dict[key]['Kingdom'])
        temp_list.append(ordered_obs_dict[key]['Phylum'])
        temp_list.append(ordered_obs_dict[key]['Class'])
        temp_list.append(ordered_obs_dict[key]['Order'])
        temp_list.append(ordered_obs_dict[key]['Family'])
        temp_list.append(ordered_obs_dict[key]['Genus'])
        temp_list.append(ordered_obs_dict[key]['Species'])
        obs_dict.update({key:{'taxonomy':temp_list}})
    biom_to_add_tax.add_metadata(obs_dict,'observation')
    return biom_to_add_tax

In [10]:
def extract_biom_table(biom_tax,kingdoms,levels,table_format,modes):
    #since kraken combined profile tables are not usefully organized, start by breaking into pieces to get the same info
    
    for level in levels:
        print("Extracting " + level)
        prefixes = tuple(levels[level][0])
        if table_format == 'Kraken':
            prefix_biom=biom_tax.filter(lambda v, i, m: i.split('|')[-1].startswith(prefixes), axis='observation', inplace=False)            
        else:
            prefix_biom = biom_tax
            
        if not level == 'Kingdom':            
            for kingdom in kingdoms:
                print("Starting "+ kingdom)                
                    
                filtered_biom=prefix_biom.filter(lambda v, i, m: m['taxonomy'][0] == kingdom, axis='observation', inplace=False).remove_empty()
                
                #clean up with low abundance filter to remove noise
                filtered_biom = low_abundance_filter(filtered_biom,0.01).remove_empty() #0.01% filter
                
                #collapse samples with metadata to remove 'ugly' format 
                bin_fn = lambda id_, x: x['taxonomy'][levels[level][1]]
                
                collapsed_biom = filtered_biom.collapse(bin_fn, norm=False, min_group_size=1,axis='observation').sort(axis='observation').remove_empty()
                
                if collapsed_biom.get_table_density() > 0:
                    for mode in modes:
                        save_biom('./merged_files/merged_' +kingdom+'_'+level + '_'+mode+'.biom', transform_table(collapsed_biom,mode))
                else:
                    print("No features selected for " + kingdom + ': ' + level)
        else:
            #collapse samples with metadata to remove 'ugly' format 
            bin_fn = lambda id_, x: x['taxonomy'][levels[level][1]]

            collapsed_biom = prefix_biom.collapse(bin_fn, norm=False, min_group_size=1,axis='observation').sort(axis='observation')
            
            if collapsed_biom.get_table_density() > 0:
                for mode in modes:
                    save_biom('./merged_files/merged_' +level + '_'+mode+'.biom', transform_table(collapsed_biom,mode))                
            else:
                print("No features selected for " + kingdom + ': ' + level)
            
def get_species(biom_tax):
    species_features =[]
    for feat in biom_tax.ids(axis='observation'):
        if feat.split('|')[-1].startswith('s__'): #assumes 's__' convention with | separators
            species_features.append(feat)
    filtered_species_biom = biom_tax.filter(species_features,axis='observation',inplace=False).remove_empty()
    return filtered_species_biom
    
def transform_table(biom_tab,mode):
    if mode =='raw':
        return biom_tab
    elif mode =='cpm':
        return biom_tab.transform(lambda data, id_, md: (data / data.sum() * 1000000).round(),axis='sample')
    elif mode =='perc':
        return biom_tab.transform(lambda data, id_, md: (data / data.sum() * 100),axis='sample')
    else:
        print('Mode ' + mode +' not found.')
        
def low_abundance_filter(input_table,percentage):
    # assumes data are in count not relative abundance
    # this creates a boolean mask where a value True indicates the feature is > 0.01%
    # and multiplies that mask against the counts (equivalent to multiplying by 0 and 1 where 0 is False and 1 is True)
    threshold = percentage/100
    filtered = input_table.transform(lambda v, i, m: v * ((v / v.sum()) >= threshold), inplace=False).remove_empty()
    return filtered

In [11]:
#central function to kick off process and load cleaned metadata and biom tables
def import_data(tuple_list,name,save_merged=False):
    #create placeholder dictionary and counter to enable merging
    pathlib.Path('./merged_files').mkdir(parents=True, exist_ok=True) 
    
    temp_dict = {}
    counter = 0
    for item in tuple_list: # list of labels, metadata, and bioms
        temp_df = clean_panda(item[1]) #create a clean, normalized metadata file
        temp_biom = clean_biom(item[2]) #load the biom table
        metadata_samples = temp_df.index.values #get sample list
        
        #drop samples for which there is no metadata and drop features that have no values
        samples_to_keep = set(metadata_samples).intersection(temp_biom.ids(axis='sample'))
        temp_biom.filter(samples_to_keep, inplace=True).filter(lambda val, _id, md: sum(val) >0,inplace=True,axis='observation')
        temp_dict.update({item[0]:{'metadata':temp_df,'biom':temp_biom}}) #add the item to the dict
        
        if counter >0:
            temp_dict.update({'merged':{'metadata':temp_dict['merged']['metadata'].append(temp_df),'biom':temp_dict['merged']['biom'].merge(temp_biom,observation='union',sample='union').remove_empty()}}) #merge the metadata
        else:
            temp_dict.update({'merged':{'metadata':temp_df,'biom':temp_biom}}) # create the merged item
            
        counter = counter + 1 #increase the counter for the loop
        
    #after completing loop write out files if requested    
    if save_merged:
        temp_dict['merged']['metadata'].to_csv('./merged_files/merged_'+name+'_metadata.tsv',index=True, index_label='sample_name',sep='\t')
        save_biom('./merged_files/merged_'+name+'.biom',temp_dict['merged']['biom'])
    return temp_dict

In [12]:
#to generate the example participant files, we'll want to update the id of choice to 'example_participant'
id_as_example = '10317.000065978'
swap_dict = {id_as_example:'10317.example_participant'}

#first kraken
base_biom = biom.load_table('./input_files/ag500_combined_profile.biom').update_ids(swap_dict,axis='sample',strict=False)
save_biom('./input_files/merged_example_kraken_table.biom',base_biom)

#now humann2
base_biom = biom.load_table('./input_files/ag500_patha.cpm.unstr.biom').update_ids(swap_dict,axis='sample',strict=False)
save_biom('./input_files/merged_example_humann2_table.biom',base_biom)

md = pd.read_csv('./input_files/ag500_metadata.tsv',header=0, sep ='\t', decimal = ',')
md.replace(id_as_example,'10317.example_participant',inplace=True)
md.to_csv('./input_files/merged_example_metadata.tsv',index=False,sep='\t')

Saved to ./input_files/merged_example_kraken_table.biom
Saved to ./input_files/merged_example_humann2_table.biom


In [14]:
#Enter the desired labels, metadata files, and biom files into the tuple list;
# of interest. This also assumes no overlap between sample_names
#should consider if/how to abstract this into command line args
tax_label_metadata_biom_tuples = [('merged_example_kraken','./input_files/merged_example_metadata.tsv','./input_files/merged_example_kraken_table.biom'),]
tax_biom_dict = import_data(tax_label_metadata_biom_tuples,True)
tax_biom_table_type = 'Kraken'

func_label_metadata_biom_tuples = [('merged_example_humann2','./input_files/merged_example_metadata.tsv','./input_files/merged_example_humann2_table.biom'),]
func_biom_dict = import_data(func_label_metadata_biom_tuples,'humann2',True)

#update with the file whose metadata will be used to generate reports
ids_to_report = ['10317.example_participant']
print("Will generate report for: " + str(ids_to_report))

#now write out metadata for individuals
save_individual_metadata(tax_biom_dict['merged']['metadata'],ids_to_report)

Saved to ./input_files/merged_example_kraken_table_positive_only.biom
Saved to ./input_files/merged_example_humann2_table_positive_only.biom
Saved to ./merged_files/merged_humann2.biom
Will generate report for: ['10317.example_participant']


In [17]:
#for humann2, just need to convert to cpm and then write out tables and then graph
function_biom_cpm = transform_table(func_biom_dict['merged']['biom'],'cpm')
function_biom_perc = transform_table(function_biom_cpm,'perc')
save_individual_bioms(ids_to_report,function_biom_cpm,'humann2_cpm')

#convert to qza and tsv and save
import_bioms(ids_to_report,'humann2_cpm')
summarize_qza(ids_to_report,'humann2_cpm')

#get top 10 pathways and bin rest as 'Other'
norm_top_df =function_biom_perc.to_dataframe()
top_level = {x: norm_top_df[x].nlargest(20).round(2) for x in norm_top_df.columns}
for participant in top_level:
                 top_level[participant] = top_level[participant].append(pd.Series({'Other':max(100-top_level[participant].sum(),0)}))

#set colors
color_palette = 'BrBG' #11 colors, divergent, color-blind aware
        
for sample in ids_to_report:
    pathlib.Path('./individual_reports/'+sample+'/static_plots/').mkdir(parents=True, exist_ok=True)
    
    sns.set()
    df = pd.DataFrame(top_level[sample])
    df.columns=['Percentage']
    ax = df.T.plot(kind='bar', stacked=True, use_index=False, title='Distribution of Functional Pathways', table=True,legend=True, colormap=color_palette)
    ax.legend(bbox_to_anchor=(1,1))
    ax.set_xmargin(1)
    ax.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    #ax.set_xlabel("Your Sample")
    ax.set_ylabel("Percentage")
    plt.savefig('./individual_reports/'+sample+'/static_plots/'+ sample + '_function.svg',bbox_inches="tight", format="svg")
    plt.close('all')
           

Saved to ./individual_reports/10317.example_participant/raw/biom/humann2_cpm/10317.example_participant_humann2_cpm_merged.biom
Saved to ./individual_reports/10317.example_participant/raw/biom/humann2_cpm/10317.example_participant_humann2_cpm_individual.biom
Imported 10317.example_participant to ./individual_reports/10317.example_participant/raw/qza/humann2_cpm/10317.example_participant_humann2_cpm_individual.qza
Summarizing 10317.example_participant to ./individual_reports/10317.example_participant/qzv/humann2_cpm/10317.example_participant_humann2_cpm_individual.qzv
Saved Visualization to: ./individual_reports/10317.example_participant/qzv/humann2_cpm/10317.example_participant_humann2_cpm_individual.qzv
finished 10317.example_participant


/home/adswafford/miniconda3/envs/qiime2-2018.4/lib/python3.5/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [18]:
kingdoms_to_extract = ['Archaea','Bacteria','Fungi','Viruses',]
levels_to_extract = {
                     'Kingdom':(['d__','k__'],0),
                     'Phylum':(['p__'],1),
                     #'Class':(['c__'],2),
                     #'Order':(['o__'],3),
                     #'Family':(['f__'],4),
                     'Genus':(['g__'],5),
                     'Species':(['s__'],6)
                    }                     
normalization_modes = ['perc',] #'cpm'=counts per million; 'perc'=percentage; swr'=sample with replacement; 'rarify'= sample without replacement

if tax_biom_dict['merged']['biom'].metadata(axis='observation') is None:
    biom_with_tax = add_taxonomy(tax_biom_dict['merged']['biom'])
    save_biom('./merged_files/merged_table_with_tax.biom',biom_with_tax)
else:
    biom_with_tax = tax_biom_dict['merged']['biom']

#parse kraken to files to be able to graph top10
extract_biom_table(biom_with_tax,kingdoms_to_extract,levels_to_extract,tax_biom_table_type,normalization_modes)

Extracting Species
Starting Archaea
Saved to ./merged_files/merged_Archaea_Species_perc.biom
Starting Bacteria
Saved to ./merged_files/merged_Bacteria_Species_perc.biom
Starting Fungi
Saved to ./merged_files/merged_Fungi_Species_perc.biom
Starting Viruses
Saved to ./merged_files/merged_Viruses_Species_perc.biom
Extracting Kingdom
Saved to ./merged_files/merged_Kingdom_perc.biom
Extracting Genus
Starting Archaea
Saved to ./merged_files/merged_Archaea_Genus_perc.biom
Starting Bacteria
Saved to ./merged_files/merged_Bacteria_Genus_perc.biom
Starting Fungi
Saved to ./merged_files/merged_Fungi_Genus_perc.biom
Starting Viruses
Saved to ./merged_files/merged_Viruses_Genus_perc.biom
Extracting Phylum
Starting Archaea
Saved to ./merged_files/merged_Archaea_Phylum_perc.biom
Starting Bacteria
Saved to ./merged_files/merged_Bacteria_Phylum_perc.biom
Starting Fungi
Saved to ./merged_files/merged_Fungi_Phylum_perc.biom
Starting Viruses
No features selected for Viruses: Phylum


In [19]:
kingdom_plot_list = ['Archaea','Bacteria','Viruses','Fungi']
level_plot_list = ['Kingdom','Phylum','Genus','Species'] #'Class','Order','Family'

In [20]:
for participant in ids_to_report:
    
    for level in level_plot_list:
        
        if level == 'Kingdom':
            p_df_kingdom =biom.load_table('./merged_files/merged_Kingdom_perc.biom').filter([participant],axis='sample').to_dataframe()

            plots_folder = './individual_reports/'+participant+'/static_plots/'
            pathlib.Path(plots_folder).mkdir(parents=True, exist_ok=True)

            sns.set()
            p_df_kingdom.columns=['Percentage']
            ax = p_df_kingdom.T.plot(kind='bar', stacked=True, use_index=False, title='Distribution of Kingdom', table=True,legend=True, colormap=color_palette)
            ax.legend(bbox_to_anchor=(1,1))
            ax.set_xmargin(1)
            ax.tick_params(
                axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom='off',      # ticks along the bottom edge are off
                top='off',         # ticks along the top edge are off
                labelbottom='off')
            ax.set_ylabel("Percentage")
            plt.savefig('./individual_reports/'+participant+'/static_plots/'+ participant + '_Kingdom.svg',bbox_inches="tight", format="svg")
            plt.close('all')
        else:
            for kingdom in kingdom_plot_list:
                #optional catch for viruses; exclude for now because few samples have family level calls
                #if kingdom == 'Viruses' and level == 'Phylum':
                #    biom_to_plot = biom.load_table('merged_Viruses_Family_perc.biom').filter([participant],axis='sample').remove_empty()
                #    plot_title = 'Distribution of Viruses Family'
                #    file_name = './individual_reports/'+participant+'/static_plots/'+ participant + '_Viruses_Family.svg'
                    
                #else:
                try:
                    biom_to_plot = biom.load_table('./merged_files/merged_' + kingdom + '_' +level + '_perc.biom').filter([participant],axis='sample').remove_empty()
                    plot_title = 'Distribution of ' +kingdom + ' ' + level
                    file_name = './individual_reports/'+participant+'/static_plots/'+ participant + '_'+kingdom+'_'+level+'.svg'
                
                    if biom_to_plot.get_table_density() > 0: #check to see that the file isn't empty
                        sns.set()
                        biom_to_plot_df = biom_to_plot.to_dataframe()
                        king_level_dict= {x: biom_to_plot_df[x].nlargest(10).round(2) for x in biom_to_plot_df.columns}
                        king_level_dict[participant]=king_level_dict[participant].append(pd.Series({'Other':max(100-biom_to_plot_df[participant].sum(),0)}))
                        king_level_df = pd.DataFrame(king_level_dict)

                        ax =king_level_df.T.plot(kind='bar', stacked=True, use_index=False, title=plot_title, table=True,legend=True, colormap=color_palette)
                        ax.legend(bbox_to_anchor=(1,1))
                        ax.set_xmargin(1)
                        ax.tick_params(
                                    axis='x',          # changes apply to the x-axis
                                    which='both',      # both major and minor ticks are affected
                                    bottom='off',      # ticks along the bottom edge are off
                                    top='off',         # ticks along the top edge are off
                                    labelbottom='off')
                        ax.set_ylabel("Percentage")
                        plt.savefig(file_name,bbox_inches="tight", format="svg")
                        plt.close('all')
                    else:
                        print("Biom table empty for " + level + " and " +kingdom)
                except:
                    print("No "+ level + " found for " + kingdom)
    
    

/home/adswafford/miniconda3/envs/qiime2-2018.4/lib/python3.5/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


No Phylum found for Viruses


In [21]:
#get species biom to use for unifrac
species_table = get_species(biom_with_tax)

#apply a low abundance filter with 0.01% cutoff and convert the table to percentage
low_pass_species = low_abundance_filter(species_table,0.01)
    
#these next three commands operate using the file structure and filenames for now
perc_suffix ='species_kraken_perc'

#write out species biom tables and .tsv version
save_individual_bioms(ids_to_report,clean_biom(low_pass_species,save=False),perc_suffix)

#import bioms as qza
import_bioms(ids_to_report,perc_suffix)

#create feature table summaries to give to participants
summarize_qza(ids_to_report,perc_suffix)

Saved to ./individual_reports/10317.example_participant/raw/biom/species_kraken_perc/10317.example_participant_species_kraken_perc_merged.biom
Saved to ./individual_reports/10317.example_participant/raw/biom/species_kraken_perc/10317.example_participant_species_kraken_perc_individual.biom
Imported 10317.example_participant to ./individual_reports/10317.example_participant/raw/qza/species_kraken_perc/10317.example_participant_species_kraken_perc_individual.qza
Summarizing 10317.example_participant to ./individual_reports/10317.example_participant/qzv/species_kraken_perc/10317.example_participant_species_kraken_perc_individual.qzv
Saved Visualization to: ./individual_reports/10317.example_participant/qzv/species_kraken_perc/10317.example_participant_species_kraken_perc_individual.qzv
finished 10317.example_participant


In [22]:
pathlib.Path('./tree').mkdir(parents=True, exist_ok=True)
create_tree(clean_biom(low_pass_species,save=False),'./tree/from_kraken_species')

taxonomy made
tree complete, starting checks


In [23]:
underscore_nwk_qza = './tree/from_kraken_species.qza'

suffix_to_plot ='species_kraken_perc'
#import individual bioms to qza and run UniFrac
print('starting UniFrac')
for sample in ids_to_report:
    qza_folder = './individual_reports/'+sample+'/raw/qza'
    
    pathlib.Path(qza_folder).mkdir(parents=True, exist_ok=True)
    
    sample_biom_table = './individual_reports/'+sample+'/raw/biom/'+suffix_to_plot+'/'+sample +'_'+suffix_to_plot+'_merged.biom'
    sample_qza = qza_folder + '/'+suffix_to_plot + '/'+sample +'_'+suffix_to_plot+'_merged.qza'
    out_dm = './individual_reports/'+sample+'/qzv/'+suffix_to_plot+ '/'+sample +'_unweighted_unifrac_distance_matrix.qza'    
        
    !qiime tools import \
        --input-path $sample_biom_table \
        --type 'FeatureTable[Frequency]' \
        --source-format BIOMV210Format \
        --output-path $sample_qza

    print("Imported, creating UniFrac plot")
        
    !qiime diversity beta-phylogenetic-alt \
        --i-table $sample_qza \
        --i-phylogeny $underscore_nwk_qza \
        --p-metric unweighted_unifrac \
        --o-distance-matrix $out_dm
        
    print("UniFrac dm made")
    
    pcoa_qza = './individual_reports/'+sample+'/qzv/'+suffix_to_plot+'/' +sample +'_unweighted_unifrac_pcoa_results.qza'
    
    !qiime diversity pcoa \
        --i-distance-matrix $out_dm \
        --o-pcoa $pcoa_qza
    
    md = './individual_reports/'+sample+'/raw/metadata/' + sample + '_metadata_merged.tsv'
    emp_plot_qzv = './individual_reports/'+sample+'/qzv/'+suffix_to_plot+'/' +sample +'_unweighted_unifrac_emperor_plot.qzv'
    
    print(emp_plot_qzv)
    
    !qiime emperor plot \
        --i-pcoa $pcoa_qza \
        --m-metadata-file $md \
        --o-visualization $emp_plot_qzv
    
    assign_emperor_settings(emp_plot_qzv)    
    
    print("Emperor made")   

starting UniFrac
Imported, creating UniFrac plot
Saved DistanceMatrix % Properties(['phylogenetic']) to: ./individual_reports/10317.example_participant/qzv/species_kraken_perc/10317.example_participant_unweighted_unifrac_distance_matrix.qza
UniFrac dm made
Saved PCoAResults to: ./individual_reports/10317.example_participant/qzv/species_kraken_perc/10317.example_participant_unweighted_unifrac_pcoa_results.qza
./individual_reports/10317.example_participant/qzv/species_kraken_perc/10317.example_participant_unweighted_unifrac_emperor_plot.qzv
Saved Visualization to: ./individual_reports/10317.example_participant/qzv/species_kraken_perc/10317.example_participant_unweighted_unifrac_emperor_plot.qzv
Emperor made
